In [9]:
%cd /content/drive/MyDrive/multi/0525

/content/drive/MyDrive/multi/0525


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 원, 사각형, 삼각형 구분하기
- 어려운 테스트 문제를 데이터 부풀리기를 이용해 데이터 개수를 늘린다.
- 기존 CNN 모델 사용

In [3]:
from google.colab import files
up = files.upload()

Saving hard_handwriting_shape.zip to hard_handwriting_shape.zip


In [3]:
!unzip ./hard_handwriting_shape.zip > /dev/null

unzip:  cannot find or open ./hard_handwriting_shape.zip, ./hard_handwriting_shape.zip.zip or ./hard_handwriting_shape.zip.ZIP.


In [4]:
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt

- 데이터셋 생성하기

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=15, # 이미지를 랜덤하게 15도 회전시킨다.
    width_shift_range=0.1, # 이미지를 수평으로 랜덤하게 이동시킨다.
    height_shift_range=0.1, # 이미지를 수직으로 랜덤하게 이동시킨다.
    zoom_range = [0.0,1.5], # 0.0 ~ 1.5배 크기로 임의로 확대/축소
    horizontal_flip = True, # 랜덤하게 이미지를 수평으로 뒤집는다.
    vertical_flip = False,  # 랜덤하게 이미지를 수직으로 뒤집는다.
    fill_mode = 'nearest'
)

In [10]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img = load_img('./handwriting/train/triangle/triangle001.png')
x = img_to_array(img)
x.shape

(24, 24, 3)

In [11]:
x = x.reshape((1,) + x.shape)
x.shape

(1, 24, 24, 3)

- 데이터 부풀리기를 통해서 만들어지는 새로운 도형 이미지

In [7]:
import os
if not os.path.exists('./preview'):
    os.mkdir('./preview')

- data generator

In [12]:
train_datagenerator = train_datagen.flow_from_directory(
    'handwriting/train', target_size=(24,24),
    batch_size=3, class_mode = 'categorical'
)

Found 45 images belonging to 3 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale=1/255.)
test_datagenerator = test_datagen.flow_from_directory(
    'handwriting/test', target_size=(24,24),
    batch_size=3, class_mode = 'categorical'
)

Found 15 images belonging to 3 classes.


- 모델 정의/설정/학습

In [14]:
import numpy as np
import tensorflow as tf
seed = 2022
tf.random.set_seed(seed)
np.random.seed(seed)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, Dense, Dropout

In [22]:
model = Sequential([
       Conv2D(32, (3,3), input_shape=(24,24,3), activation='relu'),
       Conv2D(32, (3,3), activation='relu'),
       MaxPooling2D(),
       Flatten(),
       Dense(128, activation='relu'),
       Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 22, 22, 32)        896       
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3200)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               409728    
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                      

In [23]:
from tensorflow.keras.optimizers import RMSprop
model.compile('adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
model_name = './best_hard_handwriting.h5'

mc = ModelCheckpoint(model_name, save_best_only=True)
es = EarlyStopping(patience=10)

In [25]:
model.fit(
    train_datagenerator,
    steps_per_epoch = 9,
    epochs=100,
    validation_data=test_datagenerator,
    validation_steps=5,
    callbacks=[mc,es]
)

Epoch 1/100
9/9 [==============================] - 1s 60ms/step - loss: 1.3436 - accuracy: 0.3333 - val_loss: 1.1760 - val_accuracy: 0.3333
Epoch 2/100
9/9 [==============================] - 0s 28ms/step - loss: 1.1476 - accuracy: 0.4074 - val_loss: 1.1347 - val_accuracy: 0.4000
Epoch 3/100
9/9 [==============================] - 0s 23ms/step - loss: 1.1028 - accuracy: 0.4074 - val_loss: 1.0977 - val_accuracy: 0.4000
Epoch 4/100
9/9 [==============================] - 0s 19ms/step - loss: 1.0408 - accuracy: 0.3333 - val_loss: 1.1128 - val_accuracy: 0.4000
Epoch 5/100
9/9 [==============================] - 0s 33ms/step - loss: 1.0097 - accuracy: 0.5185 - val_loss: 1.0826 - val_accuracy: 0.4667
Epoch 6/100
9/9 [==============================] - 0s 30ms/step - loss: 0.9437 - accuracy: 0.4815 - val_loss: 1.4056 - val_accuracy: 0.3333
Epoch 7/100
9/9 [==============================] - 0s 18ms/step - loss: 0.8354 - accuracy: 0.6667 - val_loss: 2.4581 - val_accuracy: 0.3333
Epoch 8/100
9/9 [===

In [26]:
model.evaluate(test_datagenerator)

5/5 [==============================] - 0s 7ms/step - loss: 2.2099 - accuracy: 0.4000


[2.209948778152466, 0.4000000059604645]